In [3]:
import sys
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
from pudl import ferc1, pudl
import numpy as np
import pandas as pd
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os.path

In [5]:
%load_ext autoreload
%autoreload 1
%aimport pudl.pudl
%aimport pudl.ferc1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
pudl_engine = create_engine('postgresql://catalyst@localhost:5432/pudl_sandbox')
ferc1_engine = create_engine('postgresql://catalyst@localhost:5432/ferc1')
pudl.pudl.PUDLBase.metadata.drop_all(pudl_engine)
pudl.pudl.PUDLBase.metadata.create_all(pudl_engine)
pudl.pudl.init_db(pudl.pudl.PUDLBase)

In [4]:
Session_pudl = sessionmaker(bind=pudl_engine)
Session_f1 = sessionmaker(bind=f1_engine)
session_pudl = Session_pudl()
session_f1 = Session_f1()

In [5]:
f1_fuel = pd.read_sql('''SELECT respondent_id, report_year, plant_name, fuel, fuel_unit, fuel_quantity,\
                                fuel_avg_heat, fuel_cost_delvd, fuel_cost_burned, fuel_cost_btu, fuel_cost_kwh,\
                                fuel_generaton \
                                FROM f1_fuel WHERE plant_name <> '' AND fuel NOT IN ('Total','') ''',f1_engine)

In [6]:
pudl_fuel = pd.read_sql('''SELECT * from fuel_ferc1''',pudl_engine)

In [12]:
# For every combnation of FERC plant, FERC utility, year, and fuel that we are interested in importing from 
# the ferc1 DB into the PUDL DB...
# select all the records in the ferc1.f1_fuel table that match that utility, plant, fuel, and year
# For each record, use the values to construct a FuelFERC1 object, and add it to the list of stuff we're going to
# instert into the pudl_sandbox.fuel_ferc1 table.
pudl_dir = os.path.dirname(os.path.dirname(pudl.__file__))
map_eia923_ferc1_file = os.path.join(pudl_dir, 'results', 'id_mapping', 'mapping_eia923_ferc1_test.xlsx')
plant_map = pd.read_excel(map_eia923_ferc1_file,'plants_output',
                              converters={'plant_id':int,
                                          'plant_name':str,
                                          'respondent_id_ferc1':int,
                                          'respondent_name_ferc1':str,
                                          'plant_name_ferc1':str,
                                          'plant_id_eia923':int,
                                          'plant_name_eia923':str,
                                          'operator_name_eia923':str,
                                          'operator_id_eia923':int})

util_map = pd.read_excel(map_eia923_ferc1_file,'utilities_output',
                             converters={'utility_id':int,
                                         'utility_name':str,
                                         'respondent_id_ferc1':int,
                                         'respondent_name_ferc1':str,
                                         'operator_id_eia923':int,
                                         'operator_name_eia923':str})

In [112]:
# For every plant_id in the plant_map
#    Get a list of utility_ids associated with that plant's respondent_id_ferc1 in the util_map
#    Get a list of utility_ids associated with that plant's operator_id_eia923 in the util_map
#    For every unique utility_id thus found, add a (plant_id, util_id) association to the table
#for x in plant_map[['plant_id','respondent_id_ferc1','operator_id_eia923']].itertuples():
#    print(x.plant_id)
#    #print(x.plant_id, x.operator_id_eia923)

plants = plant_map[['plant_id','plant_name']]
plants = plants.drop_duplicates('plant_id')

plants_eia923 = plant_map[['plant_id_eia923','plant_name_eia923','plant_id']]
plants_eia923 = plants_eia923.drop_duplicates('plant_id_eia923')

plants_ferc1 = plant_map[['plant_name_ferc1','respondent_id_ferc1','plant_id']]
plants_ferc1 = plants_ferc1.drop_duplicates(['plant_name_ferc1','respondent_id_ferc1'])

utils = util_map[['utility_id','utility_name']]
utils = utils.drop_duplicates('utility_id')

utils_eia923 = util_map[['operator_id_eia923','operator_name_eia923','utility_id']]
utils_eia923 = utils_eia923.drop_duplicates('operator_id_eia923')

utils_ferc1 = util_map[['respondent_id_ferc1','respondent_name_ferc1','utility_id']]
utils_ferc1 = utils_ferc1.drop_duplicates('respondent_id_ferc1')

plants_respondents = plant_map[['plant_id','respondent_id_ferc1']]
plants_operators = plant_map[['plant_id','operator_id_eia923']]

plant_util_ferc1 = utils_ferc1.join(plants_respondents.set_index('respondent_id_ferc1'), on='respondent_id_ferc1')
plant_util_eia923 = utils_eia923.join(plants_operators.set_index('operator_id_eia923'), on='operator_id_eia923')

plants_utils_assn = pd.concat([plant_util_eia923,plant_util_ferc1])
plants_utils_assn = plants_utils_assn[['plant_id','utility_id']].dropna().drop_duplicates()

,plant_id,utility_id
0,1.0,1
0,2.0,1
0,3.0,1
0,7.0,1
0,8.0,1
0,9.0,1
0,12.0,1
0,13.0,1
0,14.0,1
0,15.0,1


In [114]:
pudl.init_db(pudl.Base)

IntegrityError: (psycopg2.IntegrityError) insert or update on table "plants_ferc1" violates foreign key constraint "plants_ferc1_respondent_id_fkey"
DETAIL:  Key (respondent_id)=(134) is not present in table "utilities_ferc1".
 [SQL: 'INSERT INTO plants_ferc1 (respondent_id, plant_name, plant_id_pudl) VALUES (%(respondent_id)s, %(plant_name)s, %(plant_id_pudl)s)'] [parameters: ({'respondent_id': 134, 'plant_name': 'Blundell', 'plant_id_pudl': 1}, {'respondent_id': 134, 'plant_name': 'Carbon', 'plant_id_pudl': 2}, {'respondent_id': 134, 'plant_name': 'Chehalis', 'plant_id_pudl': 3}, {'respondent_id': 134, 'plant_name': 'Cholla', 'plant_id_pudl': 4}, {'respondent_id': 134, 'plant_name': 'Colstrip', 'plant_id_pudl': 5}, {'respondent_id': 134, 'plant_name': 'Craig', 'plant_id_pudl': 6}, {'respondent_id': 145, 'plant_name': 'Craig', 'plant_id_pudl': 6}, {'respondent_id': 134, 'plant_name': 'Currant Creek', 'plant_id_pudl': 7}  ... displaying 10 of 74 total bound parameter sets ...  {'respon

In [36]:
for p in plants_eia923.itertuples():
    session.add(pudl.PlantEIA923(plant_id      = int(p.plant_id_eia923),
                            plant_name    = p.plant_name_eia923,
                            plant_id_pudl = int(p.plant_id)))

In [37]:
    for p in plants_ferc1.itertuples():
        session.add(pudl.PlantFERC1(respondent_id = int(p.respondent_id_ferc1),
                               plant_name    = p.plant_name_ferc1,
                               plant_id_pudl = int(p.plant_id)))


In [38]:
for u in utils_eia923.itertuples():
    session.add(pudl.UtilityEIA923(operator_id   = int(u.operator_id_eia923),
                                   operator_name = u.operator_name_eia923,
                                   util_id_pudl  = int(u.utility_id)))


In [39]:
for u in utils_ferc1.itertuples():
    session.add(pudl.UtilityFERC1(respondent_id   = int(u.respondent_id_ferc1),
                             respondent_name = u.respondent_name_ferc1,
                             util_id_pudl    = int(u.utility_id)))

In [40]:
utils = util_map[['utility_id','utility_name']]
utils = utils.drop_duplicates('utility_id')
for u in utils.itertuples():
    session.add(pudl.Utility(id = int(u.utility_id), name = u.utility_name))


In [41]:
plants = plant_map[['plant_id','plant_name']]
plants = plants.drop_duplicates('plant_id')
for p in plants.itertuples():
    session.add(pudl.Plant(id = int(p.plant_id), name = p.plant_name))


In [42]:
session.commit()

IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "plants_pkey"
DETAIL:  Key (id)=(1) already exists.
 [SQL: 'INSERT INTO plants (id, name) VALUES (%(id)s, %(name)s)'] [parameters: ({'name': 'Dolet Hills', 'id': 1}, {'name': 'Flint Creek', 'id': 2}, {'name': 'Pirkey', 'id': 3}, {'name': '59th Street', 'id': 4}, {'name': '74th Street', 'id': 5}, {'name': 'Allen S King', 'id': 6}, {'name': 'Thousand Springs', 'id': 7}, {'name': 'A B Brown', 'id': 8}  ... displaying 10 of 7580 total bound parameter sets ...  {'name': 'CoServ Solar Station', 'id': 7580}, {'name': 'State-Fuel Level Increment', 'id': 7581})]